In [50]:
%config IPCompleter.greedy=True

In [51]:
import tensorflow as tf

In [52]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

In [53]:
import numpy as np

In [54]:
batch_size=64
epochs=10
num_samples=10000
latent_dim=256 #dimensions
data_path='fra.txt'

In [55]:
input_texts=[]
target_texts=[]
input_char= set()
target_char= set()

In [56]:
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')

In [57]:
for line in lines[: min(num_samples,len(lines)-1)]:
    input_text, target_text, _ =line.split('\t') # separating eng-fre with \t
    
    target_text1 = '\t' + target_text +'\n'  #adding start \t and end character \n in output
   
    input_texts.append(input_text)
    target_texts.append(target_text1)
    
    for char in input_text: #fetching unique input chars
        if char not in input_char:
            input_char.add(char)
            
    for char in target_text1: #fetching unique output chars
        if char not in target_char: 
            target_char.add(char)

In [58]:
input_char=sorted(list(input_char))
target_char=sorted(list(target_char))

eccharlen=len(input_char) #no of unique input tokens
dccharlen=len(target_char) #no of unique output tokens

maxslenEC=max([len(txt) for txt in input_texts])  #max seq length for input
maxslenDC=max([len(txt) for txt in target_texts]) #max seq length for output



In [59]:
inputtokenindex=dict([char,i] for i, char in enumerate(input_char))
targettokenindex=dict([char,i] for i, char in enumerate(target_char))

In [85]:
#TEST TEXT
test="Lets go"

In [117]:
encoder_inputdata=np.zeros((len(input_texts),maxslenEC,eccharlen),dtype='float32')
decoder_inputdata=np.zeros((len(target_texts),maxslenDC,dccharlen),dtype='float32')

#encode_test=np.zeros((len(test),maxslenEC,eccharlen),dtype='float32')

decoder_targetdata=np.zeros((len(target_texts),maxslenDC,dccharlen),dtype='float32')


In [97]:
#onehot repr.
for i,(input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char  in enumerate(input_text):
        encoder_inputdata[i, t, inputtokenindex[char]]=1
    encoder_inputdata[i, t+1:, inputtokenindex[' ']]=1
    
    for t, char  in enumerate(target_text):
        decoder_inputdata[i, t, targettokenindex[char]]=1
        if t>0:
            decoder_targetdata[i, t-1, targettokenindex[char]]=1
        
    decoder_inputdata[i, t+1:, targettokenindex[' ']]=1
    decoder_targetdata[i, t:, targettokenindex[' ']]=1
    


In [116]:
#encoder_inputdata[1]
#decoder_inputdata[1]
#decoder_targetdata[1]

In [63]:
#encoder model arch
encoder_inputs=Input(shape=(None,eccharlen))
encoder= LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states=[state_h,state_c]


In [64]:
#decoder model arch
decoder_inputs=Input(shape=(None,dccharlen))
decoder_lstm= LSTM(latent_dim,return_sequences=True ,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense= Dense(dccharlen, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)


In [65]:
#decoder_inputs
#decoder_outputs

In [ ]:
model= Model([encoder_inputs,decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_inputdata,decoder_inputdata], decoder_targetdata, batch_size=batch_size, epochs=epochs,validation_split=0.2 )

In [70]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, None, 94)]   0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, 256), (None, 335872      input_8[0][0]                    
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, None, 256),  359424      input_9[0][0]                    
                                                                 lstm_7[0][1]               